In [11]:
import pandas as pd
from pathlib import Path
from glob import glob
import os
from sklearn.model_selection import StratifiedKFold

In [10]:
Raw_Data = "/data/data/TCGA/lung/Raw_Data"

In [2]:
dataset = pd.read_csv("./data/lusc_group.csv")

In [ ]:
dataset

In [3]:
samples = dataset["sample"].values
risks = dataset["risk_level"].values

In [ ]:
L = []
for sample in samples:
    # print(glob("{}/{}*{}*.svs".format(Raw_Data, sample, "DX")))
    if len(glob("{}/{}*{}*.svs".format(Raw_Data, sample, "DX"))) == 0:
        continue
    l = [sample, glob("{}/{}*{}*.svs".format(Raw_Data, sample, "DX"))[0]]
    L.append(l)

f = pd.DataFrame(L, columns=['slide_id', 'slide_path'])
f.to_csv("./hs2p/data/panda.csv", index=None)

In [4]:
pt_files = "/data/data/TCGA/lung/CLAM_h5_pt/pt_files"
K = 5

In [ ]:
X  = []
y = []
y_ = []

for s,r in zip(samples, risks):
    if len(glob("{}/{}*{}*.pt".format(pt_files, s, "DX"))) == 0:
        continue
    X.append(Path(glob("{}/{}*{}*.pt".format(pt_files, s, "DX"))[0]).stem)
    y.append(1 if r == "High" else 0)
    y_.append(r)

data = [
        pd.Series(X),
        pd.Series(X), 
        pd.Series(y_)
]
df = pd.concat(data, ignore_index=True, axis=1)
df.columns = ['case_id', 'slide_id', 'label']
df.to_csv("./CLAM/dataset_csv/risk_dummy_clean.csv", index=None)

In [ ]:
skf = StratifiedKFold(n_splits=K, shuffle=True)
for fold, (train, test) in enumerate(skf.split(X, y)):
    train_data = []
    test_data = []

    train_set = pd.Series(X).iloc[train].reset_index(drop=True)
    test_set = pd.Series(X).iloc[test].reset_index(drop=True)

    df = pd.concat([train_set, test_set], ignore_index=True, axis=1)
    df.columns = ['train', 'val']
    df.to_csv(os.getcwd()+'/CLAM/splits/Clam_split_{}.csv'.format(fold), index=None)

In [14]:
for k in range(K):
    dataset = pd.read_csv("./CLAM/splits/Clam_split_{}.csv".format(k))

    data = [
        pd.Series(dataset.train),
        pd.Series(dataset.val), 
        pd.Series(dataset.val)
    ]

    df = pd.concat(data, ignore_index=True, axis=1)
    df.columns = ["train", "val", "test"]

    df.to_csv("./CLAM/splits/task_1_lung_risk_100/splits_{}.csv".format(k), index=None)

In [ ]:
python CLAM/extract_features_fp.py --data_h5_dir /data2_image_192_168_10_11/data/public_dir/xisx/20250227_JiL_lung/hs2p/output/patch_extraction/2025-02-27_17_04 --data_slide_dir /data/data/TCGA/lung/Raw_Data/ --csv_path /data2_image_192_168_10_11/data/public_dir/xisx/20250227_JiL_lung/hs2p/output/patch_extraction/2025-02-27_17_04/process_list.csv --feat_dir /data2_image_192_168_10_11/data/public_dir/xisx/20250227_JiL_lung/feat/ --model_name uni_v1